<a href="https://colab.research.google.com/github/FabioSpindola/TesteAula/blob/main/BOOTCAMP_Projeto_Porto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inicio (import Mongo, Requerimentos e Criação do banco)

## import Mongo

In [ ]:
from google.colab import userdata
MONGO_ACESSO = userdata.get("mongo_acesso")
!pip install pymongo
!pip install email_validator

## Requerimentos

In [ ]:
from pymongo import MongoClient
from pymongo.errors import PyMongoError, DuplicateKeyError
from bson import ObjectId
from bson.errors import InvalidId
from email_validator import validate_email, EmailNotValidError
from IPython.display import clear_output
from time import sleep
from enum import unique
from rich.console import Console
from rich.table import Table
from rich import box
from rich.text import Text
from rich.table import Table
from rich.console import Console
from collections import defaultdict

## Criação do banco

In [ ]:
try:
  mongo = MongoClient(MONGO_ACESSO)
  banco = mongo['Sistema_Porto_db']
  cursos = banco['cursos']
  alunos = banco['Alunos']
  avaliacoes = banco['Avaliacoes']

except PyMongoError as e:
  print(f'Erro ao conectar ao MongoDB: {e}')

cursos.create_index('titulo', unique= True)
print('tudo certo1')

alunos.create_index('email', unique= True)
print('tudo certo2')

# CRUD (Create, Read, Update, Delete)

## Estilização

In [ ]:
# linha
def linha():
  print("=" * 90)

#cabeçalho
def cabecalho(titulo="Menu"):
  linha()
  print(f" {titulo} ".center(90))
  linha()

#delay
def delay(s=1):
  sleep(s)

#limpar tela
def limpar_tela():
  clear_output()
  delay()

#pausa / volta menu
def pausa():
  input("Pressione [ENTER] para continuar\n")
  delay()

##Creates

In [ ]:
#Insert Alunos
def criar_aluno():
  while True:
    try:
      cabecalho("Criar Aluno")
      nome = input("Digite o nome do Aluno: ").strip()
      if not nome:
        print("Nome não pode ser vazio.")
        linha()
        continue

      email = input("\nDigite um email válido: ").strip()
      linha()

      verifica = ("Deseja criar o Aluno ou voltar para o menu(sair): ").strip().lower()

      if verifica == "sair":
        break

      valida = validate_email(email)

      alunos.insert_one({"nome": nome,
                      "email": email
                      })
      print("Aluno Cadastrado!")
      linha()
      break

    except DuplicateKeyError as erro:
        print("Email já cadastrado, tente outro")
    except PyMongoError as error:
        print(f"Erro ao Criar novo aluno {error}")
    except EmailNotValidError as e:
        print(f"Email Inserido não é valido!")

In [ ]:
#Insert Cursos
def criar_curso():
  while True:
    try:
      cabecalho("Criar Curso")
      titulo = input("Digite um título para o Curso: ").strip()
      categoria = input("Digite uma Categoria: ").strip()
      carga_horaria = int(input("Digite a carga horária em horas: ").strip())
      linha()

      if carga_horaria <= 0:
        print("Erro: A carga horaria deve ser maior que 0.")
        continue

      verifica = ("Deseja criar o Curso ou voltar para o menu(sair): ").strip().lower()

      if verifica == "sair":
        break

      cursos.insert_one({"titulo":titulo,
                        "categoria":categoria,
                        "carga_horaria":carga_horaria
                        })
      print("Curso Cadastrado!")
      linha()
      break

    except DuplicateKeyError as erro:
      print("Titulo já cadastrado, tente outro")
    except PyMongoError as error:
      print(f"Erro ao criar Novo curso{error}")

In [ ]:
#Insert avaliações:
def avaliar_curso():
  while True:
    cabecalho("Avaliar Curso")
    email = input("Digite o e-mail do aluno: ").strip()
    titulo = input("Digite o título do curso: ").strip()
    linha()

    if not email and not titulo:
      print("Erro: Nenhum parametro foi passado")
      break

    aluno = alunos.find_one({"email": email})
    curso = cursos.find_one({"titulo": titulo})

    if not aluno:
      print("Erro: aluno não encontrado.")
      linha()
      continue

    if not curso:
      print("Erro: curso não encontrado.")
      linha()
      continue

    verifica = ("Deseja realizar a avalição ou voltar para o menu(sair): ").strip().lower()

    if verifica == "sair":
      break

    avaliacao_existente = avaliacoes.find_one({
      "id_aluno": aluno["_id"],
      "id_curso": curso["_id"]
    })

    if avaliacao_existente:
      print("Erro: este aluno já avaliou este curso.")
      linha()
      break

    try:
      nota = int(input("Digite a nota de avaliação (1 a 5): "))
    except ValueError:
      print("Erro: nota inválida.")
      linha()
      continue

    if nota < 1 or nota > 5:
      print("Erro: a nota deve estar entre 1 e 5.")
      linha()
      continue

    comentario = input("Comentário (opcional): ")

    avaliacoes.insert_one({
      "id_aluno": aluno["_id"],
      "id_curso": curso["_id"],
      "nota": nota,
      "comentario": comentario
    })

    print(f"Avaliação registrada com sucesso para o curso '{titulo}'!")
    linha()
    break

##Reads

In [ ]:
# Listar todos os alunos cadastrados
def listar_alunos():
    try:
        if alunos.count_documents({}) == 0:
            print("Nenhum Aluno cadastrado.")
            linha()
            return

        tabela = Table(title='Alunos Cadastrados')
        tabela.add_column('Ordem',justify="center")
        tabela.add_column('ID',justify="center")
        tabela.add_column('Nome',justify="center")
        tabela.add_column('Email',justify="center")

        n = 0
        for aluno in alunos.find():

            n +=1
            ordem = str(n)
            id = str(aluno.get("_id"))
            nome = aluno.get("nome", "N/A")
            email = aluno.get("email", "N/A")
            tabela.add_row(ordem,id, nome, email)

        console = Console()
        console.print(tabela)

    except PyMongoError as e:
        print(f"Erro ao listar Alunos: {e}")

In [ ]:
# Listar todos os cursos registrados
def listar_cursos():
    try:
        if cursos.count_documents({}) == 0:
            print("Nenhum curso cadastrado.")
            linha()
            return

        tabela = Table(title='Cursos Cadastrados')
        tabela.add_column('Ordem',justify="center")
        tabela.add_column('ID',justify="center")
        tabela.add_column('Titulo',justify="center")
        tabela.add_column('Categoria',justify="center")
        tabela.add_column('Carga Horaria',justify="center")

        n = 0
        for curso in cursos.find():

            n +=1
            ordem = str(n)
            id = str(curso.get("_id"))
            titulo = curso.get("titulo")
            categoria = curso.get("categoria")
            carga_horaria = str(curso.get("carga_horaria"))
            tabela.add_row(ordem,id, titulo, categoria,carga_horaria)

        console = Console()
        console.print(tabela)

    except PyMongoError as e:
        print(f"Erro ao listar Cursos: {e}")

In [ ]:
# Listar todas as avaliaçoes por curso
def ver_avaliacoes_por_curso():
    cabecalho("Avaliações por Curso")
    titulo = input("Digite o título do curso: ").strip()
    curso = cursos.find_one({"titulo": titulo})
    linha()

    if not curso:
        print("Curso não encontrado.")
        linha()
        return

    avaliacao = avaliacoes.find({"id_curso": curso["_id"]})

    cabecalho(f"Avaliações para o curso: {titulo}")
    encontrou = False
    for av in avaliacao:
        aluno = alunos.find_one({"_id": av["id_aluno"]})
        nome_aluno = aluno["nome"] if aluno else "Desconhecido"
        print(f"⭐ {av['nota']} - {nome_aluno}")
        if av.get("comentario"):
            print(f"  Comentário: {av['comentario']}")
        encontrou = True
    linha()

    if not encontrou:
        print("Nenhuma avaliação para este curso.")

In [ ]:
#Listar avaliações feitas por aluno
def ver_avaliacoes_por_aluno():
    cabecalho("Avaliações por Aluno")
    email = input("Digite o e-mail do aluno: ").strip()
    aluno = alunos.find_one({"email": email})
    linha()

    if not aluno:
        print("Aluno não encontrado.")
        return

    avaliacao = avaliacoes.find({"id_aluno": aluno["_id"]})

    cabecalho(f"Avaliações feitas por: {aluno['nome']}")
    encontrou = False
    for av in avaliacao:
        curso = cursos.find_one({"_id": av["id_curso"]})
        nome_curso = curso["titulo"] if curso else "Curso desconhecido"
        print(f"📚 {nome_curso} - ⭐ {av['nota']}")
        if av.get("comentario"):
            print(f"  Comentário: {av['comentario']}")
        encontrou = True
    linha()

    if not encontrou:
        print("O aluno ainda não avaliou nenhum curso.")
        linha()
        return

##Updates

In [ ]:
# Atualizar Aluno:
def atualizar_aluno():
  while True:
    try:
      att = {}
      cabecalho("Atualizar Aluno")
      email = input("Informe o Email do usuario que deseja alterar:")
      valida = validate_email(email) #Valida se o email é valido
      novo_nome = input("Informe o novo nome do aluno:")
      novo_email = input("informe o novo email do aluno")
      valida_novo = validate_email(novo_email)

      # Se houver dado para alterar
      if novo_nome:
        att["nome"] = novo_nome
      if novo_email:
        att["email"] = novo_email

      # Se não houver atualização
      if not att:
        print("Nenhum dado de atualização fornecida")
        linha()
        break

      verifica = ("Deseja atualizar o aluno ou voltar para o menu(sair): ").strip().lower()

      if verifica == "sair":
        break

      resultado = alunos.update_one(
        {"email": email},
        {"$set": att}
      )

      if resultado.matched_count == 0:
        print("Nenhum aluno Cadastrado com este Email!")
        linha()
        continue
      elif resultado.modified_count == 0:
        print("Nenhuma modificação realizada.")
        linha()
        break
      else:
        print("Aluno atualizado com sucesso.")
        linha()
        break

    except DuplicateKeyError as e:
      print(f"Email '{novo_email}' já está cadastrado!")

    except PyMongoError as e:
      print(f"Erro ao atualizar Aluno: {e}")

    except InvalidId as e:
      print("ID Inválido:", id)

    except EmailNotValidError as e:
      print("E-mail inválido!")

In [ ]:
def atualizar_curso():
    while True:
        try:
            cabecalho("Atualizar Curso")
            att = {}
            titulo = input("Insira o Título do curso que deseja alterar (ou espaço para finalizar): ").strip()

            if titulo.lower() == '':
                print("Finalizando...")
                break  # Sai do loop se o usuário digitar vazio

            novo_titulo = input("Insira o novo título do curso (ou espaço para manter o mesmo): ").strip()
            nova_categoria = input("Insira a nova categoria do curso (ou espaço para manter o mesmo): ").strip()

            nova_carga_horaria = None
            while True:
                carga_horaria_input = input("Insira a nova carga-horária do curso (deixe em branco para manter a atual): ").strip()
                if not carga_horaria_input:
                    break  # Não atualizar a carga horária
                try:
                    nova_carga_horaria = int(carga_horaria_input)
                    if nova_carga_horaria <= 0:
                        print("A carga horária deve ser um número positivo. Tente novamente.")
                    else:
                        break
                except ValueError:
                    print("Carga horária inválida. Por favor, insira um número inteiro.")

            if novo_titulo:
                att["titulo"] = novo_titulo
            if nova_categoria:
                att["categoria"] = nova_categoria
            if nova_carga_horaria is not None:
                att["carga_horaria"] = nova_carga_horaria

            if not att:
                print("Nenhum dado de atualização fornecido para o curso.")
                break

            resultado = cursos.update_one(
                {"titulo": titulo},
                {"$set": att}
            )

            if resultado.matched_count == 0:
                print(f"Não foi encontrado nenhum curso com o título '{titulo}'.")
                linha()
            elif resultado.modified_count == 0:
                print("Nenhuma modificação realizada.")
                linha()
            else:
                print("Curso atualizado com sucesso!")
                linha()

        except DuplicateKeyError:
            print(f"Erro: O título '{novo_titulo}' já está cadastrado! Por favor, tente outro.")
        except PyMongoError as e:
            print(f"Erro ao atualizar Curso: {e}")
        except Exception as e:
            print(f"Ocorreu um erro: {e}")

## Deletes

In [ ]:
def deletar_aluno():
  while True:
    try:
      cabecalho("Deletar Aluno")
      opcao = input("""
      1. Deletar um Aluno
      2. Deletar todos os Alunos
      """).strip()

      if opcao == "1":
        email = input("Digite o e-mail do aluno a ser deletado: ").strip()

        aluno = alunos.find_one({"email": email})

        if not aluno:
          print("Aluno não encontrado.")
          linha()
          continue

        confirmacao = input(f"Tem certeza que deseja deletar o aluno '{aluno['nome']}' com e-mail '{email}'? (s/n): ").strip().lower()

        if confirmacao != "s":
          print("Operação cancelada.")
          linha()
          break

        alunos.delete_one({"email": email})
        print("Aluno deletado com sucesso.")
        linha()

      elif opcao == "2":
        confirmacao = input("Tem certeza que deseja deletar TODOS os alunos? (s/n): ").strip().lower()
        if confirmacao != "s":
          print("Operação cancelada.")
          linha()
          break

        alunos.delete_many({})
        print("Todos os alunos foram deletados com sucesso.")
        linha()

      else:
        print("Opção inválida.")
        linha()
        pausa()
        break

    except Exception as e:
      print(f"Erro ao deletar aluno(s): {e}")

In [ ]:
def deletar_curso():
  while True:
    try:
      cabecalho("Deletar Curso")
      opcao = input("""
      1. Deletar um Curso
      2. Deletar todos os Cursos
      """).strip()

      if opcao == "1":
        titulo = input("Digite o título do curso a ser deletado: ").strip()

        curso = cursos.find_one({"titulo": titulo})

        if not curso:
          print("Curso não encontrado.")
          linha()
          continue

        confirmacao = input(f"Tem certeza que deseja deletar o curso '{titulo}'? (s/n): ").strip().lower()

        if confirmacao != "s":
          print("Operação cancelada.")
          linha()
          break

        cursos.delete_one({"_id": curso["_id"]})
        avaliacoes.delete_many({"id_curso": curso["_id"]})
        alunos.update_many({"cursos": curso["_id"]}, {"$pull": {"cursos": curso["_id"]}})

        print("Curso e avaliações relacionadas deletados com sucesso.")
        linha()
        break

      elif opcao == "2":
        confirmacao = input("Tem certeza que deseja deletar TODOS os cursos e avaliações? (s/n): ").strip().lower()
        if confirmacao != "s":
          print("Operação cancelada.")
          linha()
          return

        cursos.delete_many({})
        avaliacoes.delete_many({})
        alunos.update_many({}, {"$set": {"cursos": []}})

        print("Todos os cursos, avaliações e vínculos com alunos foram deletados.")
        linha()
        break

      else:
        print("Opção inválida.")
        linha()
        break
    except Exception as e:
      print(f"Erro ao deletar curso(s): {e}")

##Media

In [ ]:
def medias_cursos():
  media_curso = {}

  for curso in cursos.find():
    id_curso = curso["_id"]
    titulo_curso = curso["titulo"]

    avaliacoes_curso = list(avaliacoes.find({"id_curso": id_curso}))

    if not avaliacoes_curso:
      continue

    soma = sum(av["nota"] for av in avaliacoes_curso)
    media = soma / len(avaliacoes_curso)

    media_curso[titulo_curso] = media

  return media_curso

#Desafios Opcionais

In [ ]:
# Primeiro desafio: TOP 3 CURSOS POR CATEGORIA:

def top_3_por_categoria(): # instanciação da função

    categorias = cursos.distinct("categoria") # retorna uma lista de todos os registros unicos de "categoria" na coleção de cursos,
    #Garante]indo que o TOP 3 seja calculado para cada categoria existente.

    for categoria in categorias: #loop de cada categoria unica
        ids_cursos = [c["_id"] for c in cursos.find({"categoria": categoria})] #Busca todos os cursos da ca

        pipeline = [
            {"$match": {"id_curso": {"$in": ids_cursos}}},
            {"$group": {
                "_id": "$id_curso",
                "media": {"$avg": "$nota"},
                "total": {"$sum": 1}
            }},
            {"$sort": {"media": -1, "total": -1}},
            {"$limit": 3}
        ]

        top_cursos = list(avaliacoes.aggregate(pipeline))

        cabecalho(f"Top 3 da categoria: {categoria}")
        for c in top_cursos:
            curso = cursos.find_one({"_id": c["_id"]})
            print(f" - {curso['titulo']} | Média: {c['media']:.2f} ({c['total']} avaliações)")
        linha()

In [ ]:
# Segundo desafio: RANKING USANDO A BIBLIOTECA RICH
def exibir_ranking(medias_cursos):

  console = Console()  # Cria instância da classe Console da biblioteca rich. instância usada para imprimir conteúdo formatado no terminal
  table = Table(title="Ranking de Cursos", box=box.ROUNDED, style="cyan") #criação da tabela com titulo e estilização

  # Criação de 3 colunas na tabela com Estilização
  table.add_column("Posição", justify="center", style="bold yellow")
  table.add_column("Curso", style="bold white")
  table.add_column("Média", justify="center", style="green")

  # Ordena os cursos com base em suas médias.
  ranking = sorted(medias_cursos.items(), key=lambda x: x[1], reverse=True)

  for idx, (curso, media) in enumerate(ranking, start=1):
    if idx == 1:
      curso = Text(curso, style="bold gold1")
      media = Text(str(media), style="bold gold1")
    elif idx == 2:
      curso = Text(curso, style="bold grey82")
      media = Text(str(media), style="bold grey82")
    elif idx == 3:
      curso = Text(curso, style="bold dark_orange3")
      media = Text(str(media), style="bold dark_orange3")
    elif idx <= 10:
      curso = Text(curso, style="bold white")
      media = Text(str(media), style="bold white")
    else:
      curso = Text(curso, style="dim")
      media = Text(str(media), style="dim")

    table.add_row(f"{idx}º", curso, media)

  #Imprime a tabela no terminal usando a variavel criada no inicio da função
  console.print(table)


In [ ]:
# Terceiro desafio: CÁLCULO DA MÉDIA GERAL DAS NOTAS ATRIBUIDAS POR CADA ALUNO

def media_geral():
  try:
    cabecalho("Médias de Cada Aluno")

    for aluno in alunos.find():
      id_aluno = aluno["_id"]
      nome_aluno = aluno.get("nome", "N/A")

      notas = [av["nota"] for av in avaliacoes.find({"id_aluno": id_aluno})]

      if notas:
        media = sum(notas) / len(notas)
        print(f"Aluno {nome_aluno}\n Média: {media:.2f}")
      else:
        print(f"{nome_aluno} ainda não realizou nenhuma avaliação!")
    linha()

  except Exception as erro:
    print(f"Erro: {erro}")

In [ ]:
# Quarto desafio: PESQUISA DE CURSOS POR PALAVRAS-CHAVE NO TITULO OU CATEGORIA
def filtro_curso():
    console = Console()
    try:
        while True:
           cabecalho("Filtrar Curso")
           titulo = input("Insira o Titulo do Curso que deseja buscar(Enter para Ignorar)").strip()
           categoria = input("Insira a Categoria do curso que deseja buscar(Enter para Ignorar)").strip()
           linha()

           if not titulo and not categoria:
                print("Nenhum critério fornecido.")
                break

           filtrar_cursos = {}
           if titulo:
               filtrar_cursos["titulo"] = {"$regex": titulo, "$options": "i"}  #busca parcial do titulo com options

           if categoria:
               filtrar_cursos["categoria"] = {"$regex": categoria, "$options": "i"} #busca parcial da categoria com options

           if cursos.count_documents(filtrar_cursos) == 0: #valida se existem cursos com o criterio de titulo e categoria inserido pelo usuario
               clear_output(wait=False)
               sleep(1)
               print("Nenhum curso encontrado com estes criterio, insira novamente.")
               continue


           if titulo:
               table = Table(title=f"Cursos com o titulo: {titulo}")

           if categoria:
               table = Table(title=f"Cursos com a categoria: {categoria}")

           table.add_column("ID")
           table.add_column("Titulo")
           table.add_column("Categoria")

           for curso in cursos.find(filtrar_cursos):
               id = curso.get("_id")
               nome = curso.get("titulo")
               categoria = curso.get("categoria")
               table.add_row(str(id), nome, categoria)

           console.print(table)
           break

    except PyMongoError as e:
     print(f"Erro ao buscar cursos: {e}")

In [ ]:
#GRAFICO DE DISTRIBUIÇÃO DE NOTAS DE CADA CURSO NO TERMINAL USANDO A LIB RICH

def mostrar_distribuicao_notas():
    console = Console()
    distribuicao = defaultdict(list)

    for avaliacao in avaliacoes.find():
        curso = cursos.find_one({"_id": avaliacao["id_curso"]})
        if curso:
            distribuicao[curso["titulo"]].append(avaliacao["nota"])

    for titulo, notas in distribuicao.items():
        console.rule(f"[bold green]Distribuição de Notas - {titulo}")
        notas_count = defaultdict(int)
        for nota in notas:
            notas_count[nota] += 1
        for nota in sorted(notas_count):
            barra = "█" * notas_count[nota]
            console.print(f"Nota {nota}: {barra} ({notas_count[nota]})")

In [ ]:
# - Melhorias:
#ver_avaliacoes_por_aluno() - talvez coluna
#ver_avaliacoes_por_curso() - talvez coluna
#media_geral() - talvez coluna

# - Ajustar:
#exibir_ranking() - esta dando erro por conta do parametro

# - Pendentes:
#Metodos de Excluir - aluno, curso, avaliação
#Metodos de atualizar - Aluno, curso

# talvez criar categorias padrões


# Menu

In [ ]:
def Iniciando_o_sistema():
    try:
      while True:
        limpar_tela()

        cabecalho("MENU PRINCIPAL")
        print("1. Cadastros")
        print("2. Avaliar Curso")
        print("3. Ver avaliações")
        print("4. Opções de listar")
        print("5. Atualizações")
        print("6. Deletar")
        print("0. Sair")
        linha()
        opcao = input("Escolha: ")

        if opcao == "1": # Cadastrar
          limpar_tela()
          cabecalho()
          print("1. Cadastrar curso")
          print("2. Cadastrar aluno")
          linha()
          sub_opcao = input("Escolha: ").strip()

          if sub_opcao == "1":
            limpar_tela()

            try:
                criar_curso()
                pausa()
            except ValueError:
                print("Erro: carga horária deve ser um número inteiro.")

          elif sub_opcao == "2":
            limpar_tela()

            criar_aluno()
            pausa()

          else:
            print("Opção inválida.")

        elif opcao == "2": # Avaliar
          limpar_tela()
          avaliar_curso()

          linha()
          pausa()

        elif opcao == "3": # Ver avaliações
          limpar_tela()

          cabecalho()
          print("1. Ver avaliações por curso")
          print("2. Ver avaliações por aluno")
          print("3. Ver Ranking")
          print("4. Ver Distribuição de Notas")
          linha()
          sub_opcao = input("Escolha: ").strip()

          if sub_opcao == "1":
            limpar_tela()
            ver_avaliacoes_por_curso()
            pausa()

          elif sub_opcao == "2":
            limpar_tela()
            ver_avaliacoes_por_aluno()
            pausa()

          elif sub_opcao == "3":
            limpar_tela()
            exibir_ranking(medias_cursos())  # Ranking
            pausa()

          elif sub_opcao == "4":
            limpar_tela()
            mostrar_distribuicao_notas()
            pausa()

          else:
            print("Opção inválida.")
            pausa()

        elif opcao == "4": # Opções de listar
            limpar_tela()

            cabecalho()
            print("1. Listar todos os alunos")
            print("2. Listar Todos os cursos")
            print("3. Filtrar cursos")

            linha()
            sub_opcao = input("Escolha: ").strip()

            if sub_opcao == "1":
                limpar_tela()
                listar_alunos()
                pausa()

            elif sub_opcao == "2":
              limpar_tela()
              listar_cursos()
              pausa()

            elif sub_opcao == "3":
              limpar_tela()
              filtro_curso()
              pausa()
            else:
                print("Opção inválida.")
        elif opcao == "5": # atualizações
            limpar_tela()

            cabecalho()
            print("1. Atualizar Aluno")
            print("2. Atualizar Curso")
            linha()
            sub_opcao = input("Escolha: ").strip()

            if sub_opcao == "1":
                atualizar_aluno()
                pausa()
            elif sub_opcao == "2":
                atualizar_curso()
                pausa()
            else:
              print("Opção invalida..")
              pausa()

        elif opcao == "6": # Deletar
            limpar_tela()
            cabecalho()
            print("1. Deletar Aluno")
            print("2. Deletar Curso")
            linha()
            sub_opcao = input("Escolha: ").strip()

            if sub_opcao == "1":
                deletar_aluno()
                pausa()
            elif sub_opcao == "2":
                deletar_curso()
                pausa()
            else:
                print("Opção inválida.")

        elif opcao == "0":
            limpar_tela()
            print("Saindo do sistema. Até logo!")
            break

        else:
            print("Opção inválida. Tente novamente.")
    except KeyboardInterrupt:
      print("Saindo do sistema. Volte Sempre!")

#Inciador

In [ ]:
Iniciando_o_sistema()

                                       Criar Aluno                                        
Digite o nome do Aluno: Jose Almir

Digite um email válido: jose.almir@soulcode.com
Aluno Cadastrado!
